In [ ]:
!pip install fuzzywuzzy[speedup]

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
columns_icfes = [
    'periodo',
    'cole_cod_mcpio_ubicacion',
    'cole_naturaleza',
    'cole_nombre_sede',
    'cole_nombre_establecimiento',
    'cole_cod_dane_sede',
    'cole_cod_dane_establecimiento',
    'percentil_global',
    'punt_global'
]

columns_colegio = [
    "NOMBRE_SEDE_EDUCATIVA",
    "MOMBRE_LOCALIDAD"
]

In [ ]:
data_icfes = pd.read_csv("../data/SABER11_cleaned/base_sb11.csv")
data_icfes = data_icfes[columns_icfes]

data_colegios = pd.read_excel("../data/COLEGIOS/info_colegios.xlsx")
data_colegios = data_colegios.rename(columns={"MOMBRE_LOCALIDAD": "NOMBRE_LOCALIDAD"})

In [ ]:
data_colegios.head()

In [ ]:
data_icfes.periodo.unique()

In [ ]:
data_filtered = data_icfes[
    (data_icfes['cole_cod_mcpio_ubicacion']==11001) &
    (data_icfes['periodo'].isin([20232,20242])) &
    (data_icfes["percentil_global"]>=99) &
    (data_icfes['percentil_global']<=100) &
    (data_icfes['cole_naturaleza'].isin(["OFICIAL"]))
]

In [ ]:
data_filtered.periodo.unique()

In [ ]:
# Copias de los nombres para buscar
names_to_match = data_filtered['cole_nombre_sede'].dropna().unique()
names_base = data_colegios['NOMBRE_SEDE_EDUCATIVA'].dropna().unique()

# Crear un mapeo con los matches más cercanos
matches = []
for name in names_to_match:
    match, score = process.extractOne(name, names_base, scorer=fuzz.token_sort_ratio)
    matches.append({'cole_nombre_sede': name, 'NOMBRE_SEDE_EDUCATIVA': match, 'similarity': score})

# Crear un DataFrame con los matches
match_df = pd.DataFrame(matches)

# Filtrar por un umbral de similitud (por ejemplo, >= 80)
match_df = match_df[match_df['similarity'] >= 80]

# Hacer el merge con los DataFrames originales
data_filtered_matched = pd.merge(data_filtered, match_df, on='cole_nombre_sede', how='left')
result = pd.merge(data_filtered_matched, data_colegios, on='NOMBRE_SEDE_EDUCATIVA', how='left')

In [ ]:
# Agrupar
percentil_summary = result|.groupby(
    ["periodo", "cole_nombre_establecimiento", "cole_nombre_sede", "percentil_global"]
).agg(
    {
        "cole_cod_mcpio_ubicacion": "first",
        "cole_naturaleza": "first",
        "cole_cod_dane_sede": "first",
        "cole_cod_dane_establecimiento": "first",
        "NOMBRE_LOCALIDAD": "first",
        "punt_global": "max"
    }
).reset_index()

# Reordenar columnas
ordered_columns = [
    "periodo",
    "percentil_global",
    "cole_cod_mcpio_ubicacion",
    "cole_naturaleza",
    "NOMBRE_LOCALIDAD",
    "cole_nombre_establecimiento",
    "cole_nombre_sede",
    "cole_cod_dane_sede",
    "cole_cod_dane_establecimiento",
    "punt_global"
]

percentil_summary = percentil_summary[ordered_columns]

percentil_summary.to_csv("../data/COLEGIOS/percentiles_por_colegio.csv")